# Cohere Chat on LangChain (Integration Guide)

# Cohere Chat with LangChain

In [1]:
from config.cohere import Cohere_Config
from langchain_cohere import ChatCohere
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage


cohere_config = Cohere_Config()

# Define the Cohere LLM
llm = ChatCohere(
    cohere_api_key=cohere_config.api_key,
    model=cohere_config.model_name
)

# Send a message chat without history message
current_message = [HumanMessage(content="knok knok!")]
print(llm.invoke(current_message))

# Send a chat message with chat history, note that the last message is current user message
current_message_and_history = [
    HumanMessage(content="knock knock"),
    AIMessage(content="Who's there?"),
    HumanMessage(content="Tank"),
]
print(llm.invoke(current_message_and_history))

content="Who's there?" additional_kwargs={'id': '86b1a0ee-797a-4cce-a919-d15701e25700', 'finish_reason': 'COMPLETE', 'content': "Who's there?", 'token_count': {'input_tokens': 500.0, 'output_tokens': 6.0}} response_metadata={'id': '86b1a0ee-797a-4cce-a919-d15701e25700', 'finish_reason': 'COMPLETE', 'content': "Who's there?", 'token_count': {'input_tokens': 500.0, 'output_tokens': 6.0}} id='run-d4b74b29-53bd-497d-a522-84ef1000a8c1-0' usage_metadata={'input_tokens': 500, 'output_tokens': 6, 'total_tokens': 506}
content='Tank who?' additional_kwargs={'id': '9bdf4ce4-aab6-4152-8980-f47bcc4087d5', 'finish_reason': 'COMPLETE', 'content': 'Tank who?', 'token_count': {'input_tokens': 510.0, 'output_tokens': 6.0}} response_metadata={'id': '9bdf4ce4-aab6-4152-8980-f47bcc4087d5', 'finish_reason': 'COMPLETE', 'content': 'Tank who?', 'token_count': {'input_tokens': 510.0, 'output_tokens': 6.0}} id='run-5c6b9636-b44b-45e9-bf0c-4d8b96693003-0' usage_metadata={'input_tokens': 510, 'output_tokens': 6, 

# Cohere Agents with LangChain

các tác nhân sử dụng mô hình ngôn ngữ để chọn một chuỗi hành động cần thực hiện. để sử dụng các tác nhân đa bước nhảy của cohere, hãy tạo một agent và truyền các công cụ muốn sử dụng. dưới đây là ví dụ: sử dụng công cụ tìm kiếm internet để nhận lời khuyên viết bài luận từ cohere với các trích dẫn.

In [ ]:
from langchain_cohere import ChatCohere
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from config.tavily import TavilyTool
import os

tavily_api_key = TavilyTool().tavily_api_key
cohere_api_key = Cohere_Config().api_key
cohere_model = Cohere_Config().model_name

os.environ["TAVILY_API_KEY"] = tavily_api_key # dòng này cần đưa lên trước dòng internet_search
internet_search = TavilySearchResults()

# Internet search tool - you can use any tool, and there are lots of community tools in LangChain.
# To use the Tavily tool you will need to set an API key in the TAVILY_API_KEY environment variable
# Define the cohere LLM
llm = ChatCohere(
    cohere_api_key=cohere_api_key,
    model=cohere_model,
)

# Create an agent
agent = create_cohere_react_agent(
    llm=llm,
    tools=[internet_search],
    prompt=ChatPromptTemplate.from_template("{question}")
)

# Create an agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[internet_search],
    verbose=True
)

# Generate a response
response = agent_executor.invoke({
    "question": "I want to write an essay, any tips?"
})

# See cohere's response
print(response.get("output"))
# Cohere provides exact citations for the sources it used
print(response.get("citations"))



> Entering new AgentExecutor chain...


ERROR:root:Failed to parse multihop completion for input: <|START_RESPONSE|>Plan: I will search for "essay writing tips" to gather information that can help the user write an effective essay.

Action: {"tool_name": "tavily_search_results_json", "parameters": {"query": "essay writing tips"}}O
ERROR:root:Error: Failed to parse multihop completion for input: <|START_RESPONSE|>Plan: I will search for "essay writing tips" to gather information that can help the user write an effective essay.

Action: {"tool_name": "tavily_search_results_json", "parameters": {"query": "essay writing tips"}}O


ValueError: Action Parsing Failed: <|START_RESPONSE|>Plan: I will search for "essay writing tips" to gather information that can help the user write an effective essay.

Action: {"tool_name": "tavily_search_results_json", "parameters": {"query": "essay writing tips"}}O